<a href="https://colab.research.google.com/github/tom5610/my-fsdl-text-recognizer-2022/blob/main/lab01/notebooks/c_lab01_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 01: Deep Neural Networks in PyTorch

Objective:
* how to write a basic neural network from scatch in PyTorch
* How the submodules of torch, like `torch.nn` and `torch.utils.data`, make writing performant neural network training and inference code easier

## Setup
Below cell will run full environment setup in colab.

In [72]:
import os
from pathlib import Path
current_dir = Path.cwd()

print(current_dir)

lab_idx = 1
lab_name = f"lab{str(lab_idx).zfill(2)}"
my_fsdl = "my-fsdl-text-recognizer-2022"

if current_dir.name == lab_name:
    pass 
else:
    os.chdir(f"{current_dir}/{my_fsdl}/{lab_name}")

Path.cwd()

/Users/tomlu/Workspace/my-fsdl-text-recognizer-2022/lab01


PosixPath('/Users/tomlu/Workspace/my-fsdl-text-recognizer-2022/lab01')

## Getting data and making Tensors

before we can build a model, we need data. 

Use MNIST dataset of handwritten digits.

The data used to train state-of-the-art models these days is generally too large to be stored on the disk of any single machine (to say nothing of the RAM!), so fetching data over a network is a common first step in model training.

In [73]:
 # allow 'hot-reloading' of modules
%load_ext autoreload
%autoreload 2
# needed for inline plots in some contexts
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from pathlib import Path 
import requests

def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename

data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

In [75]:
import gzip
import pickle 

def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)
    

In [76]:
import torch 

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [77]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


In [78]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [79]:
x_train.ndim, y_train.ndim

(2, 1)

In [80]:
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


In [81]:
import random 

import wandb 

import text_recognizer.metadata.mnist as metadata

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])
wandb.Image(example.reshape(*metadata.DIMS)).image

tensor(5)


## Build a DNN using only `torch.Tensor` methods and Python

### Defining the model

In [82]:
import math

import torch
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [83]:
weights.ndim, weights.shape

(2, torch.Size([784, 10]))

In [84]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [85]:
n = 3
a = torch.rand(n)
b = torch.rand(n)
print(torch.matmul(a, b))
print(a @ b)
a, b

tensor(0.9195)
tensor(0.9195)


(tensor([0.1766, 0.7578, 0.4366]), tensor([0.1783, 0.7263, 0.7731]))

In [86]:
0.5367 * 0.0799 + 0.361 * 0.3948 + 0.3556 * 0.6106

0.40253448999999997

In [87]:
a = torch.rand((3, 2))
b = torch.rand((2, 2))
result = a.mm(b)

In [88]:
a, b, result

(tensor([[0.7620, 0.8836],
         [0.3764, 0.2266],
         [0.0316, 0.5442]]),
 tensor([[0.0317, 0.9464],
         [0.3619, 0.1735]]),
 tensor([[0.3439, 0.8745],
         [0.0939, 0.3955],
         [0.1979, 0.1244]]))

In [89]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis = 1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [90]:
bs = 64 # batch size

xb = x_train[0: bs] # a batch of inputs
outs = model(xb)

print(outs[0], outs.shape)


tensor([-2.3677, -2.3937, -2.0894, -2.8503, -2.3935, -2.1285, -2.2074, -2.1515,
        -2.3162, -2.3212], grad_fn=<SelectBackward0>) torch.Size([64, 10])


## Defining the loss and metrics

In [91]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    #print('accuracy comparison - ', (preds == yb).float())
    return (preds == yb).float().mean()

In [92]:
yb = y_train[0: bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.0781)


In [93]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [94]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.3807, grad_fn=<NegBackward0>) tensor(2.3026)


In [95]:
torch.log(torch.tensor(1 / 10))

tensor(-2.3026)

In [96]:
loss = loss_func(outs, yb)

loss.backward()

In [97]:
weights.grad, bias.grad

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([-0.0234, -0.0173,  0.0399, -0.0079, -0.0544,  0.0295, -0.0021,  0.0295,
          0.0359, -0.0297]))

## Defining and running the fitting loop

We now have all the ingredients we need to fit a neural network to data:

* data (x_train, y_train)
* a network architecture with parameters (model, weights, and bias)
* a loss_func to optimize (cross_entropy) that supports .backward computation of gradients

In [98]:
lr = 0.5
epochs = 2

for epoch in range(epochs): # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad(): # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # achtung: PyTorch doesn't assume you're done with gradients
            #.         until you say so -- by explicitly "deleting" them. i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()


In [99]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(1.1104, grad_fn=<NegBackward0>) tensor(0.7344)


In [100]:
idx = random.randint(0, len(x_train))
example = x_train[idx: idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(3)


## Refactoring with core `torch.nn` components


### Using `torch.nn.functional` for stateless computation

In [101]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [102]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(1.1104, grad_fn=<NllLossBackward0>) tensor(0.7344)


### Using torch.nn.Module to define functions whose state is given by `torch.nn

In [103]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__() # this is mandatory as nn.Module does import setup
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

In [104]:
def forward(self, xb:torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()
print(model(xb)[:4])
loss = loss_func(model(xb), yb)
loss.backward()
print(model.weights.grad[::17, ::2])

tensor([[ 0.0834, -0.5464,  0.2889,  0.0533, -0.0507,  0.4940, -0.3092,  0.1692,
         -0.6899,  0.4079],
        [ 0.2463, -0.1957,  0.4502, -0.1657, -0.2697,  0.0777, -0.1134,  0.1952,
         -0.5677,  0.7868],
        [ 0.2313,  0.3113, -0.2038, -0.1581,  0.2716,  0.5513, -0.1684, -0.1080,
         -0.0918,  0.1137],
        [ 0.3737, -0.3313,  0.5608,  0.0222, -0.2071,  0.1293, -0.6640, -0.0680,
         -0.2028,  0.2594]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 3.0972e-03,  3.3074e-03,  3.1322e-03, -2.6317e-02,  1.6509e-03],
        [ 0.

In [105]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0195,  0.0107, -0.0931,  ...,  0.0036,  0.0170, -0.0368],
        [-0.0468,  0.0315, -0.0002,  ...,  0.0761, -0.0128, -0.0358],
        [-0.0209,  0.0382, -0.0646,  ...,  0.0108,  0.0632,  0.0409],
        ...,
        [-0.0134, -0.0681,  0.0277,  ...,  0.0562,  0.0365, -0.0399],
        [ 0.0830, -0.0332, -0.0410,  ..., -0.0255,  0.0277,  0.0033],
        [-0.0132, -0.0375,  0.0662,  ..., -0.0117, -0.0075,  0.0453]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [106]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs 
            end_idx = start_idx + bs
            yb = x_train[start_idx: end_idx]
            yb = y_train[start_idx: end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [107]:
print(accuracy(model(xb), yb))

tensor(0.7812)


In [108]:
import textwrap 

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")


torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [109]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__() 
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

In [110]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))

tensor(2.3278, grad_fn=<NllLossBackward0>)


In [111]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [112]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0044, -0.0330,  0.0191,  ..., -0.0279,  0.0152, -0.0124],
        [ 0.0236, -0.0334, -0.0067,  ..., -0.0039, -0.0261, -0.0018],
        [-0.0292,  0.0273,  0.0199,  ..., -0.0113, -0.0087, -0.0325],
        ...,
        [ 0.0118, -0.0124,  0.0070,  ..., -0.0002,  0.0134,  0.0352],
        [ 0.0227,  0.0122,  0.0182,  ...,  0.0352,  0.0163,  0.0142],
        [ 0.0098,  0.0185,  0.0155,  ..., -0.0161, -0.0159, -0.0347]],
       requires_grad=True)
Parameter containing:
tensor([-0.0169, -0.0308,  0.0064, -0.0216, -0.0299, -0.0196, -0.0294,  0.0288,
        -0.0056,  0.0298], requires_grad=True)


### Applying gradients with `torch.optim.Optimizer`

In [113]:
from torch import optim 

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [114]:
model = MNISTLogistic()
opt = configure_optimizer(model)

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs 
        end_idx = start_idx + bs 
        xb = x_train[start_idx: end_idx]
        yb = y_train[start_idx: end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training: ", loss_func(model(xb), yb), sep="\n\t")

after training: 
	tensor(2.3100, grad_fn=<NllLossBackward0>)


In [115]:
from text_recognizer.data.util import BaseDataset 

train_ds = BaseDataset(x_train, y_train)

In [116]:
BaseDataset??

Init signature: BaseDataset(*args, **kwds)
Source:        
class BaseDataset(torch.utils.data.Dataset):
    """Base Dataset class that simply processes data and targets through optional transforms.

    Read more: https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset

    Parameters
    ----------
    data
        commonly these are torch tensors, numpy arrays, or PIL Images
    targets
        commonly these are torch tensors or numpy arrays
    transform
        function that takes a datum and returns the same
    target_transform
        function that takes a target and returns the same
    """

    def __init__(
        self,
        data: SequenceOrTensor,
        targets: SequenceOrTensor,
        transform: Callable = None,
        target_transform: Callable = None,
    ) -> None:
        if len(data) != len(targets):
            raise ValueError("Data and targets must be of equal length")
        super().__init__()
        self.data = data
        self.targets = ta

In [117]:
model = MNISTLogistic()
opt = configure_optimizer(model)

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(2.2858, grad_fn=<NllLossBackward0>)


### Batching up data with `torch.utils.data.DataLoader`

In [118]:
from torch.utils.data import DataLoader

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size = bs)

In [119]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit 

In [120]:
model = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(0.3824, grad_fn=<NllLossBackward0>)


### Swapping in another model

In [121]:
from text_recognizer.models.mlp import MLP 

MLP.fit = fit

In [122]:
MLP.forward??

Signature: MLP.forward(self, x)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
File:      ~/Workspace/my-fsdl-text-recognizer-2022/lab01/text_recognizer/models/mlp.py
Type:      function


In [123]:
MLP.__init__??

Signature:
MLP.__init__(
    self,
    data_config: Dict[str, Any],
    args: argparse.Namespace = None,
) -> None
Docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
Source:   
    def __init__(
        self,
        data_config: Dict[str, Any],
        args: argparse.Namespace = None,
    ) -> None:

        super().__init__()
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_dim = np.prod(self.data_config["input_dims"])
        num_classes = len(self.data_config["mapping"])

        fc1_dim = self.args.get("fc1", FC1_DIM)
        fc2_dim = self.args.get("fc2", FC2_DIM)
        dropout_p = self.args.get("fc_dropout", FC_DROPOUT)

        self.fc1 = nn.Linear(input_dim, fc1_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.fc3 = nn.Linear(fc2_dim, num_classes)
File:      ~/Workspace/my-fsdl-text-recognizer-2022/lab01/text_recog

In [124]:
MLP?

Init signature: MLP(data_config: Dict[str, Any], args: argparse.Namespace = None) -> None
Docstring:      Simple MLP suitable for recognizing single characters.
Init docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
File:           ~/Workspace/my-fsdl-text-recognizer-2022/lab01/text_recognizer/models/mlp.py
Type:           type
Subclasses:     


In [125]:
digits_to_9 = list(range(10))
data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [126]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [127]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0333, -0.0181, -0.0020,  ..., -0.0313,  0.0300, -0.0191],
        [ 0.0066, -0.0238, -0.0196,  ...,  0.0259, -0.0002, -0.0045],
        [ 0.0074,  0.0105,  0.0324,  ..., -0.0196, -0.0105, -0.0287],
        ...,
        [ 0.0168,  0.0320, -0.0059,  ...,  0.0183, -0.0137,  0.0048],
        [-0.0087,  0.0025,  0.0208,  ..., -0.0119,  0.0098,  0.0350],
        [ 0.0282, -0.0066, -0.0211,  ..., -0.0160, -0.0070,  0.0144]],
       requires_grad=True)

In [128]:
%%time

print("before training: ", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training: ", loss_func(model(xb), yb))

before training:  tensor(2.3213, grad_fn=<NllLossBackward0>)
after training:  tensor(0.1146, grad_fn=<NllLossBackward0>)
CPU times: user 7.83 s, sys: 2.99 s, total: 10.8 s
Wall time: 5.9 s


## Extra goodies: data organization, validation, and acceleration


### Making the GPU go brrrr

In [134]:
mps_available = torch.backends.mps.is_available() and torch.backends.mps.is_built()

device = torch.device("cpu")
if torch.cuda.is_available():
    !nvidia-smi
    device = torch.device("cuda")
elif mps_available:
    print("Apple sillicon MPS")
    device = torch.device("mps")
else:
    print(":(")

Apple sillicon MPS


In [135]:
model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

device

device(type='mps')

In [136]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)

train_dataloader = DataLoader(train_ds, batch_size=bs)

In [137]:
for xb, yb in train_dataloader:
    print(xb.device, yb.device)
    break;

mps:0 mps:0


In [138]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)

print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(1.0923e-06, device='mps:0', grad_fn=<NllLossBackward0>)
CPU times: user 21.9 s, sys: 6.88 s, total: 28.8 s
Wall time: 22.6 s


## Adding validation data and organizing data code with `DataModule`

In [69]:
class MNISTDataModule:
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir / self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url + self.filename).content
            self.path.open("wb").write(content)

    def setup(self):
        with gzip.open(self.path, "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

        x_train, y_train, x_valid, y_valid = map(
            torch.Tensor, (x_train, y_train, x_valid, y_valid)
        )
        # print(y_train[0], y_train[0].type)
        # print(y_valid[0], y_valid[0].type)

        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_valid, y_valid, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)

    def valid_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_ds, batch_size=self.bs, shuffle=True)

In [70]:
def fit(self: nn.Module, datamodule):
    datamodule.prepare_data()
    datamodule.setup()

    val_dataloader = datamodule.valid_dataloader()

    self.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(self(xb), yb.type(torch.LongTensor).to(device)) for xb, yb in val_dataloader)

    print("before start of training: ", valid_loss / len(val_dataloader))

    opt = configure_optimizer(self)
    train_dataloader = datamodule.train_dataloader()

    for epoch in range(epochs):
        self.train()

        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb.type(torch.LongTensor).to(device))

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.eval()
        with torch.no_grad():
            valid_loss = sum(loss_func(self(xb), yb.type(torch.LongTensor).to(device)) for xb, yb in val_dataloader)
        print(epoch, valid_loss / len(val_dataloader))

MNISTLogistic.fit = fit
MLP.fit = fit

In [71]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

before start of training:  tensor(2.2361, device='mps:0')
0 tensor(23.1994, device='mps:0')
1 tensor(28.8829, device='mps:0')
2 tensor(34.7448, device='mps:0')
3 tensor(inf, device='mps:0')


KeyboardInterrupt: 

### try out different hyperparameters for the `MLP` and for training

In [ ]:
MLP.__init__??

Signature:
MLP.__init__(
    self,
    data_config: Dict[str, Any],
    args: argparse.Namespace = None,
) -> None
Docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
Source:   
    def __init__(
        self,
        data_config: Dict[str, Any],
        args: argparse.Namespace = None,
    ) -> None:

        super().__init__()
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_dim = np.prod(self.data_config["input_dims"])
        num_classes = len(self.data_config["mapping"])

        fc1_dim = self.args.get("fc1", FC1_DIM)
        fc2_dim = self.args.get("fc2", FC2_DIM)
        dropout_p = self.args.get("fc_dropout", FC_DROPOUT)

        self.fc1 = nn.Linear(input_dim, fc1_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.fc3 = nn.Linear(fc2_dim, num_classes)
File:      ~/Workspace/my-fsdl-text-recognizer-2022/lab01/text_recog

In [67]:
%%time

from argparse import Namespace 

args = Namespace(epochs=2, bs=32) 

epochs = 8
bs = 128 

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

model = MLP(data_config, args=args)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

before start of training:  tensor(2.3345, device='mps:0')
0 tensor(20.1206, device='mps:0')
1 tensor(21.7810, device='mps:0')
2 tensor(19.4818, device='mps:0')
3 tensor(8.8892, device='mps:0')
4 tensor(21.8852, device='mps:0')
5 tensor(16.5961, device='mps:0')
6 tensor(19.4235, device='mps:0')
7 tensor(19.7671, device='mps:0')
CPU times: user 1min 22s, sys: 29.4 s, total: 1min 51s
Wall time: 1min 27s


In [ ]:
val_dataloader = datamodule.valid_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0., device='mps:0')